# Funds Locked upon Exceeding Protocol Parameters

This test case illustrates that Marlowe contracts may lock funds forever if they exceed protocol parameters for script execution. The example simply tries to make too many payments in a `Close`, and because there are no branches in the contract that provide alternative to the `Close`, the funds at the script address are *locked forever*.

Without loss of generality, this example applies to any transaction-related protocol parameter:
1. Transaction size (`maxTxSize`)
2. Execution steps (`maxTxExecutionUnits.steps`)
3. Execution memory (`maxTxExecutionUnits.memory`)
4. Minimum ADA (`utxoCostPerWord`), also see ["Deposit Fails Due to Min-ADA Requirement"](min-ada.ipynb)
5. Value size (`maxValueSize`)

Also without loss of generality, this example applies to these circumstances:
1. A `Pay` may violate any of the first four protocol limits listed above, with the result that funds in the script are permanently locked.
2. A `If`, `Let`, `Assert`, or `Close` may violate any of the first three protocol limits listed above, with the result that funds in the script are permanently locked.
3. A `When` with a `Case` of `Deposit` may violate any of the five protocol limits listed above, with the result that the path for the `Case` can never be executed.
4. A `When` with a `Case` of `Choice` or `Notify` may violate any of the first three protocol limits listed above, with the result that the path for the `Case` can never be executed.

In summary, here are situations when funds may be locked permanently or when a path of execution may be blocked:

| Protocol Limit   | Protocol Parameter           | `Pay`  | `If`/`Let`/`Assert`/`Close` | `Deposit` | `Choice`/`Notify` |
|------------------|------------------------------|--------|-----------------------------|-----------|-------------------|
| Transaction size | `maxTxSize`                  | locks  | locks                       | blocks    | blocks            |
| Execution steps  | `maxTxExecutionUnits.steps`  | locks  | locks                       | blocks    | blocks            |
| Execution memory | `maxTxExecutionUnits.memory` | locks  | locks                       | blocks    | blocks            |
| Minimum ADA      | `utxoCostPerWord`            | locks  |                             | blocks    |                   |
| Value size       | `maxValueSize`               |        |                             | blocks    |                   |

👉 This example highlights the importance of performing static analysis of a Marlowe contract *before* creating the contract on the blockchain. To verify that a contract will not experience these failure mode, one needs to attempt to execute off-chain every possible path the contract could take during its complete execution or equivalently prove that execution would not violate protocol limits.

## Preliminaries

Select the network.

In [1]:
CARDANO_TESTNET_MAGIC=1567
CARDANO_NODE_SOCKET_PATH=node.socket

Select the payment credentials.

In [2]:
PAYMENT_SKEY=payment.skey
PAYMENT_VKEY=payment.vkey
PAYMENT_ADDR=$(cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file $PAYMENT_VKEY)
echo $PAYMENT_ADDR

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


View the UTxOs at the address.

In [3]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $PAYMENT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3c63978224ceda22aaa07ef2c3bbc516590e5cf4daf6468eb4ef0d4ecee2ce04     0        2303713336502 lovelace + TxOutDatumNone


Select the UTxOs for later use in the transactions.

In [4]:
TX_0=$(marlowe-cli util select --lovelace-only 10000000 $PAYMENT_ADDR | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | head)
echo $TX_0

3c63978224ceda22aaa07ef2c3bbc516590e5cf4daf6468eb4ef0d4ecee2ce04#0


## Specify the Initial State of the Contract

The contract will start with funds in 24 accounts associated with public key hashes.

Compute the public key hashes for a sequence of addresses in a wallet.

In [5]:
ROOT_PRV=william-shakespeare.root.prv

In [6]:
for i in `seq 0 24`
do
  PKH[$i]=$(
    cardano-wallet key child 1852H/1815H/0H/0/$i < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file /dev/stdout \
  | cardano-cli key verification-key --signing-key-file /dev/stdin --verification-key-file /dev/stdout \
  | cardano-cli address key-hash --payment-verification-key-file /dev/stdin \
  )
  echo "PKH(1852H/1815H/0H/0/$i) = ${PKH[$i]}"
done

PKH(1852H/1815H/0H/0/0) = 55ff7536247204d8775d1310209e32899b51b9962619f01f711c22ad
PKH(1852H/1815H/0H/0/1) = b3374bd9fbbabb477bc4747dba6c7ebc904f9689bdcacc0ea372241d
PKH(1852H/1815H/0H/0/2) = fa624f89c3796f76de9db2a96f419e22a64fce86f7edfe2fc7169423
PKH(1852H/1815H/0H/0/3) = beec84d4e2b5d3b8b5b24764e77a9cbfdda959d92f28638ff9803c67
PKH(1852H/1815H/0H/0/4) = 2439261fcb28f6d253d01b912cd00339dee520d66b7e5293a150394b
PKH(1852H/1815H/0H/0/5) = 0c31258f61fefff6e4c501040788d69bd484f397d5e3a1e15e47ecc4
PKH(1852H/1815H/0H/0/6) = 00911380b3063f7648b1cfe803b83fd02eaf1acfcbc35b35351146a3
PKH(1852H/1815H/0H/0/7) = 7dec85b5412971ace19df57f4619ab78370c76cc23c7b25a5930e803
PKH(1852H/1815H/0H/0/8) = 3bca24a78c9b50d24637b86cdabd3639c201188814f6973f8c48d456
PKH(1852H/1815H/0H/0/9) = c8719ddaadcebd7f8caa30c992309c85b22d8346699e49e468317bc5
PKH(1852H/1815H/0H/0/10) = 93e137997722293a87d6d871bf065e16e86310236de6aae66e3ad543
PKH(1852H/1815H/0H/0/11) = a9dc0e21d6b34909702ea57fa2e79df9c191aea4755340bc559aefa3
PK

Find the current time.

In [7]:
NOW=$(($(date -u +%s) * 1000))
MINUTE=$((60 * 1000))
echo $NOW

1658974431000


Each account will have 2 ADA.

In [8]:
PKH_LOVELACE=2000000

Build the state file.

In [9]:
cat > exceed-protocol-parameters.state << EOI
{
  "choices": [],
  "accounts": [
EOI

for i in `seq 0 24`
do
  if [[ i -lt 24 ]]
  then
    COMMA=,
  else
    COMMA=
  fi
  echo '    [[{"pk_hash":"'${PKH[$i]}'"},{"currency_symbol": "","token_name": ""}],'$PKH_LOVELACE']'$COMMA >> exceed-protocol-parameters.state
done

cat >> exceed-protocol-parameters.state << EOI
  ],
  "minTime": $NOW,
  "boundValues": []
}
EOI
cat exceed-protocol-parameters.state

{
  "choices": [],
  "accounts": [
    [[{"pk_hash":"55ff7536247204d8775d1310209e32899b51b9962619f01f711c22ad"},{"currency_symbol": "","token_name": ""}],2000000],
    [[{"pk_hash":"b3374bd9fbbabb477bc4747dba6c7ebc904f9689bdcacc0ea372241d"},{"currency_symbol": "","token_name": ""}],2000000],
    [[{"pk_hash":"fa624f89c3796f76de9db2a96f419e22a64fce86f7edfe2fc7169423"},{"currency_symbol": "","token_name": ""}],2000000],
    [[{"pk_hash":"beec84d4e2b5d3b8b5b24764e77a9cbfdda959d92f28638ff9803c67"},{"currency_symbol": "","token_name": ""}],2000000],
    [[{"pk_hash":"2439261fcb28f6d253d01b912cd00339dee520d66b7e5293a150394b"},{"currency_symbol": "","token_name": ""}],2000000],
    [[{"pk_hash":"0c31258f61fefff6e4c501040788d69bd484f397d5e3a1e15e47ecc4"},{"currency_symbol": "","token_name": ""}],2000000],
    [[{"pk_hash":"00911380b3063f7648b1cfe803b83fd02eaf1acfcbc35b35351146a3"},{"currency_symbol": "","token_name": ""}],2000000],
    [[{"pk_hash":"7dec85b5412971ace19df57f4619ab78370c76cc23c7

## Design the Contract

We just use the `Close` contract.

In [10]:
cat > exceed-protocol-parameters.contract << EOI
"close"
EOI
cat exceed-protocol-parameters.contract

"close"


## Create the Contract

Create the Plutus script and the initial datum.

In [11]:
marlowe-cli run initialize --contract-file exceed-protocol-parameters.contract \
                           --state-file    exceed-protocol-parameters.state    \
                           --out-file      exceed-protocol-parameters-1.marlowe

Compute the address of the Marlowe script.

In [12]:
SCRIPT_ADDR=$(jq -r .marloweValidator.address exceed-protocol-parameters-1.marlowe)
echo $SCRIPT_ADDR

addr_test1wp3v2mx0ccsh4l6kjtsa86lgnsss20f3lsgcst9jr07axpcmajgcl


Submit the transaction to create the contract.

In [13]:
TX_1=$(
marlowe-cli run execute --required-signer $PAYMENT_SKEY                         \
                        --tx-in $TX_0                                           \
                        --marlowe-out-file exceed-protocol-parameters-1.marlowe \
                        --change-address $PAYMENT_ADDR                          \
                        --out-file /dev/null                                    \
                        --submit 600                                            \
                        --print-stats                                           \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                                \
)
echo "TxId \"$TX_1\""


Fee: Lovelace 231369
Size: 1484 / 32768 = 4%
Execution units:
  Memory: 0 / 30000000 = 0%
  Steps: 0 / 10000000000 = 0%
TxId "35ee60b253518257651aac771418fbe978ba3bcb53fc37faa55ca4c2a9cae6d5"


The script address now has the contract with its initial state.

In [14]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
35ee60b253518257651aac771418fbe978ba3bcb53fc37faa55ca4c2a9cae6d5     1        50000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "19b0ee81d540c980f102381a22fb6d7644d8ebccd17d71feef4edc8330026a9c"


## Attempt to Close the Contract

Prepare the transaction to close the contract.

In [15]:
marlowe-cli run prepare --marlowe-file exceed-protocol-parameters-1.marlowe \
                        --invalid-before $NOW                      \
                        --invalid-hereafter $((NOW + 10 * MINUTE)) \
                        --out-file exceed-protocol-parameters-2.marlowe

Payment 1
  Acccount: PK "55ff7536247204d8775d1310209e32899b51b9962619f01f711c22ad"
  Payee: Party (PK "55ff7536247204d8775d1310209e32899b51b9962619f01f711c22ad")
  Ada: 2.000000
Payment 2
  Acccount: PK "b3374bd9fbbabb477bc4747dba6c7ebc904f9689bdcacc0ea372241d"
  Payee: Party (PK "b3374bd9fbbabb477bc4747dba6c7ebc904f9689bdcacc0ea372241d")
  Ada: 2.000000
Payment 3
  Acccount: PK "fa624f89c3796f76de9db2a96f419e22a64fce86f7edfe2fc7169423"
  Payee: Party (PK "fa624f89c3796f76de9db2a96f419e22a64fce86f7edfe2fc7169423")
  Ada: 2.000000
Payment 4
  Acccount: PK "beec84d4e2b5d3b8b5b24764e77a9cbfdda959d92f28638ff9803c67"
  Payee: Party (PK "beec84d4e2b5d3b8b5b24764e77a9cbfdda959d92f28638ff9803c67")
  Ada: 2.000000
Payment 5
  Acccount: PK "2439261fcb28f6d253d01b912cd00339dee520d66b7e5293a150394b"
  Payee: Party (PK "2439261fcb28f6d253d01b912cd00339dee520d66b7e5293a150394b")
  Ada: 2.000000
Payment 6
  Acccount: PK "0c31258f61fefff6e4c501040788d69bd484f397d5e3a1e15e47ecc4"
  Payee: Party (PK "0

Submit the transaction to close the contract.

In [16]:
marlowe-cli run execute --required-signer $PAYMENT_SKEY                         \
                        --tx-in "$TX_1#0"                                       \
                        --tx-in-collateral "$TX_1#0"                            \
                        --tx-in-marlowe "$TX_1#1"                               \
                        --marlowe-in-file  exceed-protocol-parameters-1.marlowe \
                        --marlowe-out-file exceed-protocol-parameters-2.marlowe \
                        --change-address $PAYMENT_ADDR                          \
                        --out-file /dev/null                                    \
                        --submit 600                                            \
                        --print-stats


Fee: Lovelace 4122863
Size: 15029 / 32768 = 45%
Execution units:
  Memory: 40971920 / 30000000 = 136%
  Steps: 12855779752 / 10000000000 = 128%
TxValidationErrorInMode (ShelleyTxValidationError ShelleyBasedEraAlonzo (ApplyTxError [UtxowFailure (WrappedShelleyEraFailure (UtxoFailure (ExUnitsTooBigUTxO (WrapExUnits {unWrapExUnits = ExUnits' {exUnitsMem' = 30000000, exUnitsSteps' = 10000000000}}) (WrapExUnits {unWrapExUnits = ExUnits' {exUnitsMem' = 40971920, exUnitsSteps' = 12855779752}}))))])) AlonzoEraInCardanoMode


: 1

The transactions fails and ***the funds in the contract are locked forever***.